In [2]:
import cycleFetch as cf
from datetime import datetime, timedelta
import pyodbc
import os
import datetime
import calendar
import json

In [3]:
# set up db connections
dev_db_conn_str = os.environ["DEV_DB_ODBC_CONN_STR"]
prod_db_conn_str = os.environ["PROD_DB_ODBC_CONN_STR"]

dev_conn = pyodbc.connect(dev_db_conn_str)
prod_conn = pyodbc.connect(prod_db_conn_str)

dev_cursor = dev_conn.cursor()
prod_cursor = prod_conn.cursor()

In [6]:
# Fetch device PMCL number from device Pmcl description
sql = '''
SELECT PmclNumber, PmclDescription FROM VW_EquipHealthPmcl WHERE 
PmclDescription LIKE '%AMSCO 400 Medium%' group by PmclNumber, PmclDescription
'''
dev_cursor.execute(sql)
pmcl_pairs = dev_cursor.fetchall()

pmcl_dict = {x[0] : x[1] for x in pmcl_pairs}


In [7]:
# Fetch Customer Device IDs based on PMCL
# Form pcml: [customer_device1, ...] hashmap
pmcl_customr_device_dict = {}
for pmcl, desc in pmcl_dict.items():

    sql = f''' 
    SELECT CustomerDevice_Id FROM CustomerDevice
    WHERE SiebelSTEEquipType = '{pmcl}'
    '''
    prod_cursor.execute(sql)
    cd_lst = prod_cursor.fetchall()
    pmcl_customr_device_dict[pmcl] = cd_lst

print(pmcl_customr_device_dict)

{'764327358/02': [(4263,), (4264,), (4310,), (4311,), (4334,), (4335,), (4422,), (4423,), (6754,), (5821,), (5822,), (5885,), (5886,), (5888,), (6078,), (6079,), (6083,), (6084,), (6902,), (6221,), (6223,), (6373,), (6374,), (6384,), (6385,), (6652,), (6653,), (7030,), (7031,), (6773,), (5559,), (5560,), (5626,), (5734,), (5736,), (5737,), (4576,), (4577,), (6842,), (4826,), (4828,), (5050,), (5294,), (5295,), (5307,), (5308,), (5312,), (5313,), (5326,), (5327,), (5330,), (5331,), (5332,), (5333,), (2255,), (2256,), (2261,), (2262,), (2349,), (2350,), (2373,), (2374,), (2409,), (6848,), (6862,), (4717,), (4718,), (699,), (700,), (701,), (819,), (820,), (6903,), (4958,), (4968,), (4969,), (4984,), (4999,), (5103,), (5104,), (5126,), (5127,), (1427,), (1428,), (1429,), (1430,), (6952,), (6646,), (6650,), (5999,), (6000,), (6001,), (6011,), (6012,), (4395,), (4405,), (5209,), (5210,), (586,), (587,), (588,), (646,), (647,), (6251,), (6252,), (6253,), (6254,), (6255,), (6297,), (2463,), (2

In [8]:
# fetch cycle id, starttime, endtime,
# Form device id: (cycle id, starttime, endtime) hashmap
# [{'pmcl': pmcl, 'devices': [{'deviceId': deviceId, 'cycle_ids': [{id, starttime, endtime, duration}]}]}]#

# return object in the following format:
# [{'pmcl': pmcl, 
#   'devices':[
#      {'deviceId': deviceId,
#       'cycle_ids': [{
#           'cycleId': id,
#           'starttime': starttime,
#           'endtime': endtime,
#           'duration': duration
#       }]}
#  ]}]

def add_cycle_data(pmcl_customer_device_dict):

    full_list = []
    cycle_not_in_connectcarereporting = []
    for pmcl, cd_list in pmcl_customr_device_dict.items():
        obj = {'pmcl': pmcl, 'devices': [] }

        for customer_device_id in cd_list:
            device = {}
            customer_device_id = customer_device_id[0]

            sql = f""" 
            SELECT DeviceCycle_Id, CycleStartDateTime, CycleEndDateTime, CycleType FROM UCBCycleData
            WHERE CustomerDeviceId = '{customer_device_id}' ORDER BY CycleStartDateTime ASC
            """
            sql2 = f""" 
            SELECT DeviceCycle_Id, CycleStartDateTime, CycleEndDateTime, CycleType FROM UCBCycleDataSecondary
            WHERE CustomerDeviceId = '{customer_device_id}' ORDER BY CycleStartDateTime ASC
            """

            prod_cursor.execute(sql)
            cycle_ids = prod_cursor.fetchall()
            if cycle_ids == []:
                # print(f"Device {customer_device_id}'s cycle data is not in the first table")
                prod_cursor.execute(sql2)
                cycle_ids = prod_cursor.fetchall()
            if cycle_ids == []:
                cycle_not_in_connectcarereporting.append(customer_device_id)

            else:
                cycle_ids_dict_list = []
                for row in cycle_ids:
                    row = list(row)
                    row[1] = calendar.timegm(row[1].timetuple())
                    row[2] = calendar.timegm(row[2].timetuple())
                    d = {"cycleId": row[0], "starttime": row[1], "endtime": row[2], 'duration':row[2]-row[1], 'type': row[3]}
                    cycle_ids_dict_list.append(d)
                    device['deviceId'] = customer_device_id
                    device['cycle_ids'] = cycle_ids_dict_list
            if len(device) != 0:
                obj['devices'].append(device)

        if obj['devices'] != []:
            full_list.append(obj)
    return full_list


In [9]:
full_list = add_cycle_data(pmcl_customr_device_dict)
print(full_list[0]['devices'][0]['cycle_ids'])
# Monitor which machines do not have cycle data in connectcarereporting
# with open("no_cycle_data.txt", 'w') as file:
#     for data in cycle_not_in_connectcarereporting:
#         file.write("%s\n" % data)

[{'cycleId': 12433368.0, 'starttime': 1646315016, 'endtime': 1646315987, 'duration': 971, 'type': 'DART WARMUP'}, {'cycleId': 12465041.0, 'starttime': 1646703543, 'endtime': 1646704641, 'duration': 1098, 'type': 'DART WARMUP'}, {'cycleId': 12465175.0, 'starttime': 1646704808, 'endtime': 1646706867, 'duration': 2059, 'type': 'LEAK TEST'}, {'cycleId': 12465308.0, 'starttime': 1646707802, 'endtime': 1646709651, 'duration': 1849, 'type': 'DART TEST'}, {'cycleId': 12465454.0, 'starttime': 1646710177, 'endtime': 1646711824, 'duration': 1647, 'type': 'DART TEST'}, {'cycleId': 12465654.0, 'starttime': 1646712715, 'endtime': 1646714373, 'duration': 1658, 'type': 'DART TEST'}, {'cycleId': 12466226.0, 'starttime': 1646714874, 'endtime': 1646719189, 'duration': 4315, 'type': 'PREVAC'}, {'cycleId': 12466488.0, 'starttime': 1646719617, 'endtime': 1646722925, 'duration': 3308, 'type': 'PREVAC'}, {'cycleId': 12471020.0, 'starttime': 1646765081, 'endtime': 1646769168, 'duration': 4087, 'type': 'PREVAC'

In [10]:
# Write to a json file which is then ready for further preprocessing

with open("with_cycle_data.json", "w") as file:
    json.dump(full_list, file)  # encode dict into JSON
print("Done writing dict into .json file")


Done writing dict into .json file


In [17]:
with open('cycleIDs_by_machine.json', 'r') as file:
    text = json.load(file)

for obj in text:
    devices = obj['devices']
    for device in devices:
        cycles = device['cycle_ids']
        for cycle in cycles:

            print(cf.fetch_cycle_json(int(cycle['cycleId']))['cycles'][0]['cycle_type'])
            break
        break

DART WARMUP
DART WARMUP


In [4]:
# returns true if the alarm is during a cycle
# returns false if the alarm is in between cycles
def during(cycle, alarm_ts):
    return alarm_ts <= cycle['endtime'] and alarm_ts >= cycle['starttime']


# add alarm labels to each cyle 
def insert_alarm_to_cycle(device, alarms):
    cycle_ids = device['cycle_ids']
    alarms_num = len(alarms)
    cycle_nums = len(cycle_ids)
    end_times = [x['endtime'] for x in cycle_ids]
    alarms_ts = [x['alarm_ts'] for x in alarms]

    for alarm_ts in alarms_ts:

        last = True
        not_during = True
        for i in range(len(cycle_ids)):
            if (during(cycle_ids[i], alarm_ts)):
                print("condition 1 met")
                t = cycle_ids[i]['type']
                acc = 20
                j = i
                while j > -1 and acc > 0:
                    if cycle_ids[j]['type'] == t:
                        cycle_ids[j]['FSR Required'] = 1
                        acc -= 1
                    j -= 1
                # cycle_ids[i]['FSR Required'] = 1
                not_during = False
                last = False
                break
        
        if not_during:
            for i in range(len(cycle_ids)):
                if (alarm_ts - cycle_ids[i]['endtime'] < 0):
                     print('condition 2 met')
                     acc = 20
                     t = cycle_ids[i-1]['type']
                     j = i - 1
                     while j > -1 and acc > 0:
                        if cycle_ids[j]['type'] == t:
                            cycle_ids[j]['FSR Required'] = 1
                            acc -= 1
                        j -= 1
                    #  cycle_ids[i-1]['FSR Required'] = 1
                     last = False
                     break
        if last:
            print("condition 3 met")
            # for i in range(len(cycle_ids)-1 , len(cycle_ids)-11, - 1):
            t = cycle_ids[-1]['type']
            acc = 20
            j = len(cycle_ids)-1
            while (acc > 0 and j > -1):
                if cycle_ids[j]['type'] == t:
                    cycle_ids[j]['FSR Required'] = 1
                    acc -= 1
                j -= 1



In [5]:
# test for alarm addition

with open('test.json', 'r') as file:
    device = json.load(file)
alarms = [{"alarmLogId": 99999, 'alarm_ts':221}]
insert_alarm_to_cycle(device, alarms)
print(device)


condition 1 met
{'deviceId': 4422, 'cycle_ids': [{'cycleId': 12433368.0, 'starttime': 0, 'endtime': 10, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465041.0, 'starttime': 20, 'endtime': 30, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 40, 'endtime': 50, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 60, 'endtime': 70, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 80, 'endtime': 90, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 100, 'endtime': 110, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 120, 'endtime': 130, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 140, 'endtime': 150, 'duration': 10, 'type': 'PREVAC', 'FSR Required': 1}, {'cycleId': 12465175.0, 'starttime': 160, 'endtime': 170, 'duratio

In [6]:
# retrieve alarm data, only for 'FSR Required' alarms, aka alarm priority = 1
# add alarm data to corresponding device data

# form a new json using cycle data (with alarm)
def add_alarm_data(full_list):
    counter = 0
    for obj in full_list:
        devices = obj['devices']
        for device in devices:
            device_id = device['deviceId']
            print(f'Device {device_id} alarming adding initiated')
            alarm_lst = []
            sql = f''' 
            SELECT Created_Date, CustomerDeviceAlarmLog_Id 
            FROM CustomerDeviceAlarmLog WHERE CustomerDeviceId = '{device_id}'
            AND AlarmPriority = 1 ORDER BY Created_Date ASC
            '''
            prod_cursor.execute(sql)
            alarms = prod_cursor.fetchall()
            if (len(alarms) == 0):
                continue
            for alarm in alarms:
                counter += 1
                row = list(alarm)
                row[0] = calendar.timegm(row[0].timetuple())
                d = {"alarmLogId": row[1], 'alarm_ts':row[0]}
                alarm_lst.append(d)
                break
            insert_alarm_to_cycle(device, alarm_lst)
    print(f'{counter} alarms added')
    return full_list
        


In [7]:
with open('with_cycle_data.json', 'r') as file:
    text = json.load(file)
full_data = add_alarm_data(text)

with open("full_data.json", "w") as file:
    json.dump(full_data, file) 
print("Done writing dict into .json file")


Device 4422 alarming adding initiated
condition 2 met
Device 4423 alarming adding initiated
condition 2 met
Device 6079 alarming adding initiated
Device 6083 alarming adding initiated
Device 6084 alarming adding initiated
condition 3 met
Device 5559 alarming adding initiated
condition 2 met
Device 4826 alarming adding initiated
condition 2 met
Device 4828 alarming adding initiated
condition 2 met
Device 5294 alarming adding initiated
condition 2 met
Device 5295 alarming adding initiated
condition 2 met
Device 5307 alarming adding initiated
condition 2 met
Device 5308 alarming adding initiated
condition 2 met
Device 5312 alarming adding initiated
condition 2 met
Device 5313 alarming adding initiated
condition 1 met
Device 5330 alarming adding initiated
condition 2 met
Device 2349 alarming adding initiated
condition 2 met
Device 2350 alarming adding initiated
condition 2 met
Device 2373 alarming adding initiated
Device 2374 alarming adding initiated
condition 2 met
Device 4717 alarming a

In [4]:
with open("full_data.json", 'r') as file:
    pmcl1 = json.load(file)
    pmcl1 = pmcl1[1]
with open("pmcl_1.json", 'w') as file:
    json.dump(pmcl1, file)